In [1]:
import numpy as np
import pandas as pd

# 상권정보 데이터 전처리

In [2]:
df = pd.read_csv("소상공인시장진흥공단_상가(상권)정보_서울_202106.csv", sep=",")

In [3]:
pd.set_option('display.max_seq_items', None)

In [4]:
df.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,17174175,비지트,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1165010100108540018009586,상랑의빌딩,서울특별시 서초구 동광로18길 82,137837,6572.0,NaN,1,NaN,126.991394,37.488375
1,17174119,쓰리에프,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1159010700101390073009536,NaN,서울특별시 동작구 동작대로27가길 12,156816,7008.0,NaN,NaN,NaN,126.980952,37.487105
2,17174096,채움,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1111012400100580000017956,두산위브파빌리온,서울특별시 종로구 삼봉로 81,110858,3150.0,NaN,NaN,NaN,126.981794,37.572387
3,17174062,호구의주방,NaN,D,소매,D07,가정/주방/인테리어,D07A17,주방가구판매,G47520,...,1156012800101600002019768,약산상가아파트,서울특별시 영등포구 선유로 269,150867,7206.0,NaN,NaN,NaN,126.897892,37.536700
4,17174040,다향,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1165010800114850004022127,NaN,서울특별시 서초구 효령로 230,137869,6709.0,NaN,1,NaN,127.009382,37.483436


In [5]:
df['행정동명']

0                방배4동
1                사당2동
2         종로1.2.3.4가동
3                양평2동
4                서초3동
             ...     
316073           홍은2동
316074            연남동
316075           성내3동
316076          신대방1동
316077            보광동
Name: 행정동명, Length: 316078, dtype: object

In [66]:
df.columns

Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도'],
      dtype='object')

In [12]:
# 음식점 데이터만 쓸 겁니다
df = df.loc[df['상권업종대분류명'] == '음식']  

# 다음과 같은 칼럼만 있으면 됩니다
df = df[['상호명', '상권업종중분류명', '상권업종소분류명', '표준산업분류명', '행정동명', '위도', '경도']]

# 그 중에서도 흑석동과 상도1동만 쓸 겁니다.
df = df.loc[(df['행정동명'] == '흑석동') | (df['행정동명'] == '상도1동')]

In [13]:
# 칼럼명 단순화

df.columns = ['name',  # 상호명
              'cate_1',  # 중분류명
              'cate_2',  # 소분류명
              'cate_3',  # 표준산업분류명
              'dong',  # 행정동명
              'lon',  # 위도
              'lat'  # 경도
              ]

In [17]:
df['cate_mix'] = df['cate_1'] + df['cate_2'] +  df['cate_3']
df['cate_mix'] = df['cate_mix'].str.replace("/", " ")

# 상권정보 데이터 코사인 유사도

In [25]:
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도


count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(df['cate_mix'].values.astype('U'))
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

In [29]:
df['cate_mix']

2140      닭 오리요리후라이드 양념치킨치킨 전문점
2614            유흥주점호프 맥주기타 주점업
3450         일식 수산물곰장어전문한식 음식점업
6254             제과제빵떡케익제과점제과점업
7129      닭 오리요리후라이드 양념치킨치킨 전문점
                  ...          
312054       한식한식 백반 한정식한식 음식점업
312886                      NaN
314142          한식갈비 삼겹살한식 음식점업
314171       한식부대찌개 섞어찌개한식 음식점업
315215       닭 오리요리삼계탕전문한식 음식점업
Name: cate_mix, Length: 350, dtype: object

# 네이버 가게 url 크롤링

In [19]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time


chromedriver = 'chromedriver' 
driver = webdriver.Chrome(chromedriver) 

# 포스팅 작성 당시 크롬 버젼 : 92


# 네이버 지도 검색창에 [~동 @@식당]으로 검색해 정확도를 높여야 합니다. 검색어를 미리 설정해줍시다.

df['naver_keyword'] = df['dong'] + "%20" + df['name']  # "%20"는 띄어쓰기를 의미합니다.
df['naver_map_url'] = ''


# 본격적으로 가게 상세페이지의 URL을 가져옵시다

for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(3.5)
        df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()


# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다

df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']


# URL이 수집되지 않은 데이터는 제거합니다.
df = df.loc[~df['naver_map_url'].isnull()]

이번에 찾을 키워드 : 0 / 647 행 흑석동%20Bloom
이번에 찾을 키워드 : 1 / 647 행 흑석동%20노랑통닭
이번에 찾을 키워드 : 2 / 647 행 흑석동%20파티파티노래주점
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 3 / 647 행 흑석동%20맥주창고
이번에 찾을 키워드 : 4 / 647 행 흑석동%20Mr.꼼장어Miss쭈꾸미
이번에 찾을 키워드 : 5 / 647 행 상도1동%20가춘
이번에 찾을 키워드 : 6 / 647 행 상도1동%20BHC치킨
이번에 찾을 키워드 : 7 / 647 행 상도1동%20대나무골
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 8 / 647 행 상도1동%20빈털털이
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_inf

이번에 찾을 키워드 : 53 / 647 행 흑석동%20아람중화요리
이번에 찾을 키워드 : 54 / 647 행 흑석동%20큰맘원조할매순대국
이번에 찾을 키워드 : 55 / 647 행 흑석동%20우뇽파스타뚝배기스파게티
이번에 찾을 키워드 : 56 / 647 행 흑석동%20롯데식당흑석동점
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 57 / 647 행 흑석동%20호치킨하우스
이번에 찾을 키워드 : 58 / 647 행 흑석동%20장원아구찜감자탕
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 59 / 647 행 상도1동%20벨라의케이크
이번에 찾을 키워드 : 60 / 647 행 상도1동%20친구
이번에 찾을 키워드 : 61 / 647 행 흑석동%20대박회집
이번에 찾을 키워드 : 62 / 647 행 상도1동%20낭만치맥
이번에 찾을 키워드 : 63 / 647 행 상도1동%20맘스터치
이번에 찾을 키워드 : 64 / 647 행 흑석동%20술광장호프치킨
이번에 찾을 키워드 : 65 / 647 행 흑석동%20삼백돈돈가츠
이번에

이번에 찾을 키워드 : 125 / 647 행 상도1동%20토모니베이커리
이번에 찾을 키워드 : 126 / 647 행 상도1동%20우리보쌈&밥상
이번에 찾을 키워드 : 127 / 647 행 흑석동%20슈퍼피맥
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 128 / 647 행 상도1동%20투썸플레이스숭실대점
이번에 찾을 키워드 : 129 / 647 행 흑석동%20한우동
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 130 / 647 행 상도1동%20추억과김밥
이번에 찾을 키워드 : 131 / 647 행 상도1동%20지코바양념치킨
이번에 찾을 키워드 : 132 / 647 행 상도1동%20구이와찌게
이번에 찾을 키워드 : 133 / 647 행 상도1동%20보그너커피상도동점
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > l

이번에 찾을 키워드 : 185 / 647 행 상도1동%20커피만
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 186 / 647 행 상도1동%20쿠시야끼하루
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 187 / 647 행 상도1동%20청정베이커리
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 188 / 647 행 상도1동%20구백냥곱창
이번에 찾을 키워드 : 189 / 647 행 상도1동%20알렉스플레이스
이번에 찾을 키워드 : 190 / 647 행 흑석동%20커리야CURRYYA
Mess

이번에 찾을 키워드 : 249 / 647 행 상도1동%20도담도담
이번에 찾을 키워드 : 250 / 647 행 흑석동%20박군하누
이번에 찾을 키워드 : 251 / 647 행 상도1동%20바른탕수육
이번에 찾을 키워드 : 252 / 647 행 상도1동%20임종예밥집
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 253 / 647 행 상도1동%20투다리
이번에 찾을 키워드 : 254 / 647 행 상도1동%20어블리
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 255 / 647 행 흑석동%20사랑방
이번에 찾을 키워드 : 256 / 647 행 흑석동%20중앙돼지마을
이번에 찾을 키워드 : 257 / 647 행 상도1동%20다가미
이번에 찾을 키워드 : 258 / 647 행 상도1동%20미니밈케이크
이번에 찾을 키워드 : 259 / 647 행 상도1동%20걸작떡볶이
이번에 찾을 키워드 : 260 / 647 행 흑석동%20구름산추어탕
이번에 찾을 키워드 : 261 / 647 행 상도1동%20프렌밀리
이번에 찾을 키워드

이번에 찾을 키워드 : 332 / 647 행 흑석동%20아딸
이번에 찾을 키워드 : 333 / 647 행 상도1동%20정통왕족발
이번에 찾을 키워드 : 334 / 647 행 상도1동%20싸움의고수
이번에 찾을 키워드 : 335 / 647 행 상도1동%20오딧세이
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 336 / 647 행 흑석동%20신전떡볶이중앙대점
이번에 찾을 키워드 : 337 / 647 행 상도1동%20다락
이번에 찾을 키워드 : 338 / 647 행 흑석동%20스시톡톡
이번에 찾을 키워드 : 339 / 647 행 흑석동%20장수
이번에 찾을 키워드 : 340 / 647 행 상도1동%20제이엠
이번에 찾을 키워드 : 341 / 647 행 상도1동%20팔팔구
이번에 찾을 키워드 : 342 / 647 행 흑석동%20육반연반
이번에 찾을 키워드 : 343 / 647 행 상도1동%20카페디차
이번에 찾을 키워드 : 344 / 647 행 상도1동%20닭치고
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 345 /

이번에 찾을 키워드 : 400 / 647 행 흑석동%20봉추찜닭중앙대점
이번에 찾을 키워드 : 401 / 647 행 상도1동%20굽네치킨
이번에 찾을 키워드 : 402 / 647 행 상도1동%20갓프레소
이번에 찾을 키워드 : 403 / 647 행 상도1동%20프라이밋
이번에 찾을 키워드 : 404 / 647 행 흑석동%20압구정봉구비어
이번에 찾을 키워드 : 405 / 647 행 상도1동%20오니기리와이규동숭실대점
이번에 찾을 키워드 : 406 / 647 행 흑석동%20빽다방중앙대병원점
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 407 / 647 행 상도1동%20이레김밥
이번에 찾을 키워드 : 408 / 647 행 흑석동%20퍼블릭하우스
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 409 / 647 행 상도1동%20밀짚모자
이번에 찾을 키워드 : 410 / 647 행 상도1동%20알로하치킨
이번에 찾을 키워드 : 411 / 647 행 흑석동%20상도늘보리흑석점
이번에 찾을 키워드 : 412 / 647 행 상

이번에 찾을 키워드 : 475 / 647 행 흑석동%20프랑세즈
이번에 찾을 키워드 : 476 / 647 행 상도1동%20청진동해장국
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 477 / 647 행 상도1동%20착한피자
이번에 찾을 키워드 : 478 / 647 행 상도1동%20컵오브컴포트
이번에 찾을 키워드 : 479 / 647 행 흑석동%20브라운필투
이번에 찾을 키워드 : 480 / 647 행 흑석동%20설빙
이번에 찾을 키워드 : 481 / 647 행 흑석동%20재팔이네닭발
이번에 찾을 키워드 : 482 / 647 행 흑석동%20맥피스
이번에 찾을 키워드 : 483 / 647 행 상도1동%20또오리
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 484 / 647 행 상도1동%20형제상회
이번에 찾을 키워드 : 485 / 647 행 상도1동%20부어치킨
이번에 찾을 키워드 : 486 / 647 행 흑석동%20카페복희씨
이번에 찾을 키워드 : 487 / 647 행 상도1동%20배달삼겹직구삼
이번에 찾을 키워드

이번에 찾을 키워드 : 539 / 647 행 흑석동%20비온후
이번에 찾을 키워드 : 540 / 647 행 상도1동%20전주콩나물해장국전문
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 541 / 647 행 흑석동%20아몬드블루
이번에 찾을 키워드 : 542 / 647 행 흑석동%20수미락
이번에 찾을 키워드 : 543 / 647 행 상도1동%20용구비어
이번에 찾을 키워드 : 544 / 647 행 흑석동%20고기먹으러
이번에 찾을 키워드 : 545 / 647 행 상도1동%20이모네집
이번에 찾을 키워드 : 546 / 647 행 상도1동%20마루스시
이번에 찾을 키워드 : 547 / 647 행 흑석동%20박광호닭강정&족발지존
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 548 / 647 행 흑석동%20윤정즉석김밥
이번에 찾을 키워드 : 549 / 647 행 상도1동%20상도정마루
이번에 찾을 키워드 : 550 / 647 행 흑석동%20피자스쿨
이번에 찾을 키워드 : 551 / 647 행 흑석동%20현주네김밥
이번

이번에 찾을 키워드 : 622 / 647 행 흑석동%20아츠
이번에 찾을 키워드 : 623 / 647 행 상도1동%20선녀호두파이
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 624 / 647 행 상도1동%20배스킨라빈스31
이번에 찾을 키워드 : 625 / 647 행 상도1동%20태리로제떡볶이&닭강정
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=92.0.4515.131)

이번에 찾을 키워드 : 626 / 647 행 상도1동%20카페오드리
이번에 찾을 키워드 : 627 / 647 행 흑석동%20컴포즈커피
이번에 찾을 키워드 : 628 / 647 행 상도1동%20종로계림닭도리탕원조
이번에 찾을 키워드 : 629 / 647 행 흑석동%20깐부치킨
이번에 찾을 키워드 : 630 / 647 행 상도1동%20오갈미
이번에 찾을 키워드 : 631 / 647 행 흑석동%20백소정
이번에 찾을 키워드 : 632 / 647 행 상도1동%20매머드익스프레스
이번에 찾을 키워드 : 633 / 647 행 흑석동%20이공커피
이번에 찾을 키워드 : 634 / 647 행 흑석동%20랭

In [21]:
from tqdm import tqdm_notebook

# 네이버 가게 평점, 블로그 리뷰 크롤링

In [23]:
# 각 데이터들을 미리 리스트에 담은 다음, 마지막에 데이터 프레임에 합칠 것입니다.

naver_map_type_list = []
blog_review_list = []
blog_review_qty_list = []
naver_map_star_review_stars_list = []
naver_map_star_review_qty_list = []
chromedriver = 'chromedriver' 

# 메인 드라이버 : 별점 등을 크롤링
driver = webdriver.Chrome(chromedriver) 

# 서브 드라이버 : 블로그 리뷰 텍스트를 리뷰 탭 들어가서 크롤링
sub_driver = webdriver.Chrome(chromedriver)

for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):

    driver.get(url)
    sub_driver.get(url+"/review/ugc")
    time.sleep(2)


    try:

        # 간단 정보 가져오기
        
        # 네이버 지도의 유형 분류
        naver_map_type = driver.find_element_by_css_selector("#_title > span._3ocDE").text

        # 블로그 리뷰 수
        blog_review_qty = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(3) > a > em").text

        # 블로그 별점 점수
        star_review_stars = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span._1Y6hi._1A8_M > em").text

        # 블로그 별점 평가 수
        star_review_qty = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(2) > a > em").text
       

        # 블로그 리뷰 텍스트 가져오기
        review_text_list = [] # 임시 선언

        
        # 네이버 지도 블로그 리뷰 탭은 동적 웹사이트의 순서가 주문하기, 메뉴보기 등의 존재 여부로 다르기 때문에 css selector가 아니라 element 찾기로 진행
        review_text_crawl_list = sub_driver.find_elements_by_class_name("_2CbII")
        
        # find element's' 메소드를 통해 가져온 내용은 리스트로 저장되고, 리스트 타입을 풀어서(for문 사용) 임시 데이터에 모아 두어야 한다
        for review_crawl_data in review_text_crawl_list:
            review_text_list.append(review_crawl_data.find_element_by_tag_name('div').text)
        
        # 그 리스트에 저장된 텍스트 (한 식당에 대한 여러 리뷰들)를 한 텍스트 덩어리로 모아(join)줍니다.
        review_text = ','.join(review_text_list)


        blog_review_list.append(review_text)

        naver_map_type_list.append(naver_map_type)
        blog_review_qty_list.append(blog_review_qty)
        naver_map_star_review_stars_list.append(star_review_stars)
        naver_map_star_review_qty_list.append(star_review_qty)

    # 리뷰가 없는 업체는 크롤링에 오류가 뜨므로 표기해둡니다.
    except Exception as e1:
        print(f"{i}행 문제가 발생")
        
        # 리뷰가 없으므로 null을 임시로 넣어줍니다.
        blog_review_list.append('null')  
        naver_map_type_list.append('null')
        blog_review_qty_list.append('null')
        naver_map_star_review_stars_list.append('null')
        naver_map_star_review_qty_list.append('null')


        
driver.quit()
sub_driver.quit()


df['naver_store_type'] = naver_map_type_list  # 네이버 상세페이지에서 크롤링한 업체 유형
df['naver_star_point'] = naver_map_star_review_stars_list  # 네이버 상세페이지에서 평가한 별점 평점
df['naver_star_point_qty'] = naver_map_star_review_qty_list  # 네이버 상세페이지에서 별점 평가를 한 횟수
df['naver_blog_review_txt'] = blog_review_list  # 네이버 상세페이지에 나온 블로그 리뷰 텍스트들
df['naver_blog_review_qty'] = blog_review_qty_list  # 네이버 상세페이지에 나온 블로그 리뷰의 총 개수

C:\Users\tkdal\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


  0%|          | 0/442 [00:00<?, ?it/s]

0행 문제가 발생
6행 문제가 발생
19행 문제가 발생
21행 문제가 발생
22행 문제가 발생
24행 문제가 발생
30행 문제가 발생
33행 문제가 발생
39행 문제가 발생
42행 문제가 발생
47행 문제가 발생
59행 문제가 발생
65행 문제가 발생
68행 문제가 발생
73행 문제가 발생
76행 문제가 발생
83행 문제가 발생
84행 문제가 발생
102행 문제가 발생
112행 문제가 발생
121행 문제가 발생
124행 문제가 발생
127행 문제가 발생
129행 문제가 발생
130행 문제가 발생
137행 문제가 발생
142행 문제가 발생
143행 문제가 발생
151행 문제가 발생
152행 문제가 발생
155행 문제가 발생
173행 문제가 발생
176행 문제가 발생
177행 문제가 발생
178행 문제가 발생
179행 문제가 발생
180행 문제가 발생
182행 문제가 발생
184행 문제가 발생
187행 문제가 발생
188행 문제가 발생
190행 문제가 발생
191행 문제가 발생
192행 문제가 발생
194행 문제가 발생
195행 문제가 발생
196행 문제가 발생
201행 문제가 발생
206행 문제가 발생
209행 문제가 발생
215행 문제가 발생
217행 문제가 발생
224행 문제가 발생
229행 문제가 발생
260행 문제가 발생
268행 문제가 발생
282행 문제가 발생
290행 문제가 발생
294행 문제가 발생
296행 문제가 발생
297행 문제가 발생
319행 문제가 발생
325행 문제가 발생
335행 문제가 발생
343행 문제가 발생
349행 문제가 발생
350행 문제가 발생
352행 문제가 발생
353행 문제가 발생
357행 문제가 발생
358행 문제가 발생
360행 문제가 발생
361행 문제가 발생
365행 문제가 발생
371행 문제가 발생
378행 문제가 발생
380행 문제가 발생
385행 문제가 발생
395행 문제가 발생
403행 문제가 발생
411행 문제가 발생
414행 문제가 발생
417행 문제가 발생
418행 문제가 발생
419행 문제가 발생


In [35]:
df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']]

,naver_star_point,naver_star_point_qty,naver_blog_review_qty
2140,4.45,121,25
2614,4.5,17,5
3450,4.64,18,16
6254,4.54,369,66
7129,4.47,195,9
...,...,...,...
312054,4.56,40,6
312886,4.5,10,5
314142,4.45,90,19
314171,4.31,37,11


In [33]:
# 크롤링 에러가 떠서 'null'을 넣어 둔 데이터는 활용 의미가 없으므로 행 삭제를 해줘도 됩니다
df = df.loc[~(df['naver_store_type'].str.contains('null'))]

In [37]:
# 별점 평균, 수 같은 데이터 역시 스트링 타입으로 크롤링이 되었으므로 numeric으로 바꿔줍니다.
df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']] = df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']].apply(pd.to_numeric, errors='coerce')

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 2140 to 315215
Data columns (total 15 columns):
name                     350 non-null object
cate_1                   350 non-null object
cate_2                   350 non-null object
cate_3                   338 non-null object
dong                     350 non-null object
lon                      350 non-null float64
lat                      350 non-null float64
cate_mix                 338 non-null object
naver_keyword            350 non-null object
naver_map_url            350 non-null object
naver_store_type         350 non-null object
naver_star_point         350 non-null float64
naver_star_point_qty     329 non-null float64
naver_blog_review_txt    350 non-null object
naver_blog_review_qty    350 non-null int64
dtypes: float64(4), int64(1), object(10)
memory usage: 43.8+ KB


# 리뷰데이터 코사인 유사도

In [40]:
# 리뷰 텍스트 데이터 간의 텍스트 피쳐 벡터라이징
count_vect_review = CountVectorizer(min_df=2, ngram_range=(1,2))
place_review = count_vect_review.fit_transform(df['naver_blog_review_txt']) 

# 리뷰 텍스트 간의 코사인 유사도 따지기
place_simi_review = cosine_similarity(place_review, place_review)
place_simi_review_sorted_ind = place_simi_review.argsort()[:, ::-1]

In [56]:
# 공식 1~5의 중요성을 짬뽕시키는 공식
# * 0.003 등의 가중치를 줘서 조절합니다.

place_simi_co = (
                 + place_simi_cate * 0.3 # 공식 1. 카테고리 유사도
                 + place_simi_review * 1 # 공식 2. 리뷰 텍스트 유사도
                 + np.repeat([df['naver_blog_review_qty'].values], len(df['naver_blog_review_qty']) , axis=0) * 0.001  # 공식 3. 블로그 리뷰가 얼마나 많이 올라왔는지
                 + np.repeat([df['naver_star_point'].values], len(df['naver_star_point']) , axis=0) * 0.005            # 공식 4. 블로그 별점이 얼마나 높은지
                 + np.repeat([df['naver_star_point_qty'].values], len(df['naver_star_point_qty']) , axis=0) * 0.001    # 공식 5. 블로그 별점 평가가 얼마나 많이 됐는지
                 )



# 아래 place_simi_co_sorted_ind 는 그냥 바로 사용하면 됩니다.
place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 


# 최종 구현 함수
def find_simi_place(df, sorted_ind, place_name, top_n=10):
    
    place_title = df[df['name'] == place_name]
    place_index = place_title.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]


# 상도국수를 포함해 5개 업체를 뽑아봅시다.

find_simi_place(df, place_simi_co_sorted_ind, '상도국수', 5)

,index,name,cate_1,cate_2,cate_3,dong,lon,lat,cate_mix,naver_keyword,naver_map_url,naver_store_type,naver_star_point,naver_star_point_qty,naver_blog_review_txt,naver_blog_review_qty
283,253621,매머드커피중앙대점,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,흑석동,37.507741,126.960572,커피점 카페커피전문점 카페 다방비알콜 음료점업,흑석동%20매머드커피중앙대점,https://m.place.naver.com/restaurant/1566784937,카페,4.40,NaN,흑석동 중앙대학교 부속 병원앞의 매머드커피 매장입니다. 매머드 이름과 로고 때문에 ...,5
261,224860,형제상회,한식,해장국/감자탕,한식 음식점업,상도1동,37.502311,126.948670,한식해장국 감자탕한식 음식점업,상도1동%20형제상회,https://m.place.naver.com/restaurant/32812007,생선회,4.43,NaN,"업장에서 인스타 맞팔한 분들의 게시물을 보다 단골 회센터이자 전국적 명성, 우리나라...",625
67,77570,래미안베이커리,제과제빵떡케익,제과점,제과점업,상도1동,37.497432,126.953251,제과제빵떡케익제과점제과점업,상도1동%20래미안베이커리,https://m.place.naver.com/restaurant/11834678,베이커리,4.36,NaN,"안녕하세요. 10번째 뚜레쥬르, 뚜레쥬르 상도래미안점 포스팅입니다. 비가 갑자기 쏟...",10
298,275467,스타벅스,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,상도1동,37.503228,126.948431,커피점 카페커피전문점 카페 다방비알콜 음료점업,상도1동%20스타벅스,https://m.place.naver.com/restaurant/37391902,카페,4.47,NaN,숭실대입구역 스타벅스 공부하기 좋아요 ! 배배공은 비록 옆학교를 다니지만 제가 자주...,71
297,275344,엠,유흥주점,룸살롱/단란주점,일반유흥 주점업,흑석동,37.508494,126.962145,유흥주점룸살롱 단란주점일반유흥 주점업,흑석동%20엠,https://m.place.naver.com/restaurant/31144138,미용실,4.91,NaN,헤어아티스트M 중앙대점 설 연휴 잘 보내고 있으신가요 ! 모두 새해 복 많이 받으세...,145


In [55]:
# index 재정의 (이게 어디 line에 들어가야하지)
df = df.reset_index()

# 데이터 저장

In [58]:
df.to_csv("naver_review_0810.csv", index=False)

In [60]:
pd.read_csv("naver_review_0810.csv")

,index,name,cate_1,cate_2,cate_3,dong,lon,lat,cate_mix,naver_keyword,naver_map_url,naver_store_type,naver_star_point,naver_star_point_qty,naver_blog_review_txt,naver_blog_review_qty
0,2140,노랑통닭,닭/오리요리,후라이드/양념치킨,치킨 전문점,흑석동,37.507850,126.960672,닭 오리요리후라이드 양념치킨치킨 전문점,흑석동%20노랑통닭,https://m.place.naver.com/restaurant/38419604,"치킨,닭강정",4.45,121.0,서울 흑석동 중앙대 치맥 맛집. 노랑통닭 순살 3종 세트 맛있어요. 2020년 9월...,25
1,2614,맥주창고,유흥주점,호프/맥주,기타 주점업,흑석동,37.508109,126.961404,유흥주점호프 맥주기타 주점업,흑석동%20맥주창고,https://m.place.naver.com/restaurant/31040500,"맥주,호프",4.50,17.0,흑석동 맥주창고에서 간단하게 맥주한잔!!! 퇴근하고 친구가 간단하게 한잔하자고해서 ...,5
2,3450,Mr.꼼장어Miss쭈꾸미,일식/수산물,곰장어전문,한식 음식점업,흑석동,37.508488,126.962541,일식 수산물곰장어전문한식 음식점업,흑석동%20Mr.꼼장어Miss쭈꾸미,https://m.place.naver.com/restaurant/12880938,"해물,생선요리",4.64,18.0,흑석동 중앙대 맛집 미스터꼼장어 미스쭈꾸미 얼마 전 처음으로 ! 꼼장어를 먹어보게 ...,16
3,6254,가춘,제과제빵떡케익,제과점,제과점업,상도1동,37.496929,126.953495,제과제빵떡케익제과점제과점업,상도1동%20가춘,https://m.place.naver.com/restaurant/1123429741,베이커리,4.54,369.0,상도곱창 바로 옆에 있는 #빵집 #가춘 딱봐도 #맛집 느낌이길래 상도사는 친구에게 ...,66
4,7129,BHC치킨,닭/오리요리,후라이드/양념치킨,치킨 전문점,상도1동,37.494928,126.957436,닭 오리요리후라이드 양념치킨치킨 전문점,상도1동%20BHC치킨,https://m.place.naver.com/restaurant/37413677,"치킨,닭강정",4.47,195.0,bhc 포테킹 후라이드 신메뉴 가격 요기요앱에서 할인 중 ! @bhc 일요일은 거의...,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,312054,유부랑뚱이랑,한식,한식/백반/한정식,한식 음식점업,상도1동,37.495272,126.953620,한식한식 백반 한정식한식 음식점업,상도1동%20유부랑뚱이랑,https://m.place.naver.com/restaurant/1067754092,한식,4.56,40.0,11시까지 만나기로 햇스면서 유유미 자기가 지각함 그래서 점순 유미 기다리느라 메가...,6
346,312886,타이거분식,분식,라면김밥분식,NaN,흑석동,37.507347,126.958813,NaN,흑석동%20타이거분식,https://m.place.naver.com/restaurant/1209154771,종합분식,4.50,10.0,집 앞에 귀여운 분식집이 생겼다 호기심에 한 번 방문 깔끔한 인테리어 호랑이 관련된...,5
347,314142,배달삼겹직구삼동작직영점,한식,갈비/삼겹살,한식 음식점업,상도1동,37.499935,126.951033,한식갈비 삼겹살한식 음식점업,상도1동%20배달삼겹직구삼동작직영점,https://m.place.naver.com/restaurant/1124696525,돼지고기구이,4.45,90.0,안녕하세요 : D 혼자 장보고 차려먹기는 귀찮고.. 며칠 전부터 계속 구운 고기가 ...,19
348,314171,찌개대학부대과,한식,부대찌개/섞어찌개,한식 음식점업,상도1동,37.494785,126.956613,한식부대찌개 섞어찌개한식 음식점업,상도1동%20찌개대학부대과,https://m.place.naver.com/restaurant/20590408,"찌개,전골",4.31,37.0,"안녕하세요. 김상훈의 창업통입니다. 대학가상권 음식점 사장의 즐거움, 아시나요? 서...",11
